In [ ]:
import random
import time
import decimal
from decimal import Decimal, ROUND_HALF_UP
from z3 import Real, Solver, If, sat
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from fractions import Fraction



def compute_corporate_income_tax_z3(
    OperatingRevenueTotal_input=0, SalesReturn_input=0, SalesAllowance_input=0,
    OperatingCost_input=0, OperatingExpensesLosses_input=0,
    NonOperatingRevenueTotal_input=0, NonOperatingLossExpenses_input=0,
    Prev10LossDeduction_input=0, TaxIncentiveExempt_input=0,
    ExemptSecuritiesIncome_input=0, ExemptLandIncome_input=0,
    Article4_4HouseLandGain_input=0,
    is_full_year=True,    # 營業期間是否滿一年
    m_partial=12          # 營業期間未滿一年時，營業月份數
):
    s = Solver()

    # --- 1. 定義 12 個變數 ---
    OperatingRevenueTotal    = Real('OperatingRevenueTotal')
    SalesReturn              = Real('SalesReturn')
    SalesAllowance           = Real('SalesAllowance')
    OperatingCost            = Real('OperatingCost')
    OperatingExpensesLosses  = Real('OperatingExpensesLosses')
    NonOperatingRevenueTotal = Real('NonOperatingRevenueTotal')
    NonOperatingLossExpenses = Real('NonOperatingLossExpenses')
    Prev10LossDeduction      = Real('Prev10LossDeduction')
    TaxIncentiveExempt       = Real('TaxIncentiveExempt')
    ExemptSecuritiesIncome   = Real('ExemptSecuritiesIncome')
    ExemptLandIncome         = Real('ExemptLandIncome')
    Article4_4HouseLandGain  = Real('Article4_4HouseLandGain')

    s.add(OperatingRevenueTotal    == OperatingRevenueTotal_input)
    s.add(SalesReturn              == SalesReturn_input)
    s.add(SalesAllowance           == SalesAllowance_input)
    s.add(OperatingCost            == OperatingCost_input)
    s.add(OperatingExpensesLosses  == OperatingExpensesLosses_input)
    s.add(NonOperatingRevenueTotal == NonOperatingRevenueTotal_input)
    s.add(NonOperatingLossExpenses == NonOperatingLossExpenses_input)
    s.add(Prev10LossDeduction      == Prev10LossDeduction_input)
    s.add(TaxIncentiveExempt       == TaxIncentiveExempt_input)
    s.add(ExemptSecuritiesIncome   == ExemptSecuritiesIncome_input)
    s.add(ExemptLandIncome         == ExemptLandIncome_input)
    s.add(Article4_4HouseLandGain  == Article4_4HouseLandGain_input)

    # --- 2. 中間計算變數 ---
    OperatingRevenueNet   = Real('OperatingRevenueNet')
    OperatingGrossProfit  = Real('OperatingGrossProfit')
    OperatingNetProfit    = Real('OperatingNetProfit')
    YearlyIncome          = Real('YearlyIncome')
    T                     = Real('T')

    s.add(OperatingRevenueNet  == OperatingRevenueTotal - SalesReturn - SalesAllowance)
    s.add(OperatingGrossProfit == OperatingRevenueNet - OperatingCost)
    s.add(OperatingNetProfit   == OperatingGrossProfit - OperatingExpensesLosses)
    s.add(YearlyIncome         == OperatingNetProfit + NonOperatingRevenueTotal - NonOperatingLossExpenses)

    # --- 3. 分支計算應納稅額 T ---
    P = Real('P')
    s.add(P == YearlyIncome 
        - Prev10LossDeduction 
        - TaxIncentiveExempt 
        - ExemptSecuritiesIncome 
        - ExemptLandIncome 
        - Article4_4HouseLandGain)
    
    if is_full_year:
        T_expr = If(P <= 120000,
                    0,
                    If(P <= 200000,
                       (P - 120000) * 0.5,
                       P * 0.20))
    else:
        P_adj = Real('P_adj')
        s.add(P_adj == P * 12 / m_partial)
        T_expr = If(P_adj <= 120000,
                    0,
                    If(P_adj <= 200000,
                       (P_adj - 120000) * 0.5 * (m_partial / 12),
                       P_adj * 0.20 * (m_partial / 12)))
    piecewise_T = Real('piecewise_T')
    s.add(piecewise_T == T_expr)
    s.add(T == piecewise_T)

    if s.check() == sat:
        model = s.model()
        # 如果模型返回的 T 是一個有理數，則可以用 numerator_as_long 和 denominator_as_long
        r = model[T]
        try:
            num = r.numerator_as_long()
            den = r.denominator_as_long()
            frac = Fraction(num, den)
        except Exception:
            # 如果沒有這個方法，則退回 as_decimal 轉換 (但通常有理數可以這麼取)
            r_str = r.as_decimal(50)
            if r_str.endswith('?'):
                r_str = r_str[:-1]
            frac = Fraction(r_str)
        # 這裡我們依照需求直接取整（即捨去小數部分，不進行四捨五入）
        return int(frac)
    else:
        raise ValueError("Z3 無法求解或約束衝突。")


# ========== Selenium 測試自動化 ==========
def run_corporate_tax_test():
    
    for i in range(10):
        print(f"===== 迴圈第 {i+1} 次 =====")
        # 隨機產生參數（皆為整數）
        while True:
            OperatingRevenueTotal_input    = random.randint(1_000_000, 10_000_000)
            SalesReturn_input              = random.randint(0, 500_000)
            SalesAllowance_input           = random.randint(0, 200_000)
            OperatingCost_input            = random.randint(1_000_000, 8_000_000)
            OperatingExpensesLosses_input  = random.randint(0, 3_000_000)
            NonOperatingRevenueTotal_input = random.randint(0, 1_000_000)
            NonOperatingLossExpenses_input = random.randint(0, 500_000)
            Prev10LossDeduction_input      = random.randint(0, 1_000_000)
            TaxIncentiveExempt_input       = random.randint(0, 100_000)
            ExemptSecuritiesIncome_input   = random.randint(0, 100_000)
            ExemptLandIncome_input         = random.randint(0, 100_000)
            Article4_4HouseLandGain_input  = random.randint(0, 50_000)

            # 隨機決定是否滿一年
            is_full_year = random.choice([True, False])
            if is_full_year:
                select_val = "Y"
                m_partial = 12
            else:
                select_val = "N"
                m_partial = random.randint(1, 11)


            # 計算 YearlyIncome 和 P
            OperatingRevenueNet = OperatingRevenueTotal_input - SalesReturn_input - SalesAllowance_input
            OperatingGrossProfit = OperatingRevenueNet - OperatingCost_input
            OperatingNetProfit = OperatingGrossProfit - OperatingExpensesLosses_input
            YearlyIncome = OperatingNetProfit + NonOperatingRevenueTotal_input - NonOperatingLossExpenses_input
            P = YearlyIncome - Prev10LossDeduction_input - TaxIncentiveExempt_input - ExemptSecuritiesIncome_input - ExemptLandIncome_input - Article4_4HouseLandGain_input

            if YearlyIncome > 0 and P > 0:
                break  # 符合條件，跳出迴圈

        # 取得本地計算結果
        local_tax = compute_corporate_income_tax_z3(
            OperatingRevenueTotal_input, SalesReturn_input, SalesAllowance_input,
            OperatingCost_input, OperatingExpensesLosses_input,
            NonOperatingRevenueTotal_input, NonOperatingLossExpenses_input,
            Prev10LossDeduction_input, TaxIncentiveExempt_input,
            ExemptSecuritiesIncome_input, ExemptLandIncome_input,
            Article4_4HouseLandGain_input,
            is_full_year=is_full_year,
            m_partial=m_partial
        )
        print(f"本地計算結果: {local_tax}", "參數:", {
            "OperatingRevenueTotal": OperatingRevenueTotal_input,
            "SalesReturn": SalesReturn_input,
            "SalesAllowance": SalesAllowance_input,
            "OperatingCost": OperatingCost_input,
            "OperatingExpensesLosses": OperatingExpensesLosses_input,
            "NonOperatingRevenueTotal": NonOperatingRevenueTotal_input,
            "NonOperatingLossExpenses": NonOperatingLossExpenses_input,
            "Prev10LossDeduction": Prev10LossDeduction_input,
            "TaxIncentiveExempt": TaxIncentiveExempt_input,
            "ExemptSecuritiesIncome": ExemptSecuritiesIncome_input,
            "ExemptLandIncome": ExemptLandIncome_input,
            "Article4_4HouseLandGain": Article4_4HouseLandGain_input,
            "is_full_year": is_full_year,
            "m_partial": m_partial
        })    

if __name__ == "__main__":
    run_corporate_tax_test()


===== 迴圈第 1 次 =====
本地計算結果: 40479 參數: {'OperatingRevenueTotal': 8183506, 'SalesReturn': 359540, 'SalesAllowance': 39200, 'OperatingCost': 4565660, 'OperatingExpensesLosses': 2710390, 'NonOperatingRevenueTotal': 370206, 'NonOperatingLossExpenses': 146789, 'Prev10LossDeduction': 339778, 'TaxIncentiveExempt': 70410, 'ExemptSecuritiesIncome': 82483, 'ExemptLandIncome': 23170, 'Article4_4HouseLandGain': 13894, 'is_full_year': True, 'm_partial': 12}
===== 迴圈第 2 次 =====
本地計算結果: 858584 參數: {'OperatingRevenueTotal': 8601796, 'SalesReturn': 51728, 'SalesAllowance': 70039, 'OperatingCost': 2362417, 'OperatingExpensesLosses': 1670357, 'NonOperatingRevenueTotal': 567055, 'NonOperatingLossExpenses': 443280, 'Prev10LossDeduction': 119652, 'TaxIncentiveExempt': 24460, 'ExemptSecuritiesIncome': 86706, 'ExemptLandIncome': 35722, 'Article4_4HouseLandGain': 11567, 'is_full_year': True, 'm_partial': 12}
===== 迴圈第 3 次 =====
本地計算結果: 0 參數: {'OperatingRevenueTotal': 8196285, 'SalesReturn': 289440, 'SalesAllowa